#### Import standard Python modules

In [1]:
import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib

#### Import Azure ML SDK modules

In [2]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

#### Check Azure ML SDK version

In [3]:
print(azureml.core.VERSION)

1.0.10


#### Create Azure ML Workspace

In [4]:
AZ_SUBSCRIPTION_ID='902c97d2-65e2-4835-9793-997867b2df19'
ws = Workspace.create(name='salary',
                      subscription_id=AZ_SUBSCRIPTION_ID, 
                      resource_group='mi2',
                      create_resource_group=True,
                      location='southeastasia'
                     )

Falling back to use azure cli credentials. This fall back to use azure cli credentials will be removed in the next release. 
Make sure your code doesn't require 'az login' to have happened before using azureml-sdk, except the case when you are specifying AzureCliAuthentication in azureml-sdk.


#### Write configuration to local file

In [5]:
ws.write_config()

Wrote the config file config.json to: C:\Users\saurabh.keshari\Tech_Utsav\AzureML_Tutorial\azureml-tutorial-master\Azure\aml_config\config.json


#### Create Azure ML Experiment

In [6]:
exp = Experiment(workspace=ws, name='salexp')

#### Start logging metrics

In [7]:
run = exp.start_logging()                   
run.log("Experiment start time", str(datetime.datetime.now()))

#### Load salary dataset

In [8]:
sal = pd.read_csv('data/sal.csv',header=0, index_col=None)
X = sal[['x']]
y = sal['y']

#### Split the train and test data

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

#### Train the model

In [10]:
lm = LinearRegression() 
lm.fit(X_train,y_train) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

#### Freeze the model

In [11]:
filename = 'outputs/sal_model.pkl'
joblib.dump(lm, filename)

['outputs/sal_model.pkl']

#### Test the model

In [13]:
filename = 'outputs/sal_model.pkl'
loaded_model=joblib.load(filename)
y=loaded_model.predict([[22]])[0]
print(y)

143600.7452574526


#### Log metrics to Azure ML Experiment

In [14]:
run.log('Intercept :', lm.intercept_)
run.log('Slope :', lm.coef_[0])

#### End Azure ML Experiment

In [15]:
run.log("Experiment end time", str(datetime.datetime.now()))
run.complete()

#### Get Portal URL

In [16]:
print(run.get_portal_url())

https://mlworkspace.azure.ai/portal/subscriptions/902c97d2-65e2-4835-9793-997867b2df19/resourceGroups/mi2/providers/Microsoft.MachineLearningServices/workspaces/salary/experiments/salexp/runs/336c3a34-d016-472d-a139-22684e18c9d4


Here now 2nd phase ends. We can go to the above url to see all the experiment run results

From here 3rd phase starts: Register the Model

#### Register the model

In [17]:
model = Model.register(model_path = "outputs/sal_model.pkl",
                       model_name = "sal_model",
                       tags = {"key": "1"},
                       description = "Salary Prediction",
                       workspace = ws)

Registering model sal_model


#### Define Azure ML Deploymemt configuration

#pkl file donot help much
#we need to go beyond it
#We now tell Azure that take model and run it in Azure container instance which is one container model.

In [18]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Salary",  "method" : "sklearn"}, 
                                               description='Predict Stackoverflow Salary')

#### Create enviroment configuration file

# We also need to tell AzureML what to do with Model
#2 things our model will do:
#1. A runtime env- then model is serialised - A web service has to figure out that what needs to do with it.
We need to explain the dependencies like numpy, sklearn, etc

In [19]:
salenv = CondaDependencies()
salenv.add_conda_package("scikit-learn")

with open("salenv.yml","w") as f:
    f.write(salenv.serialize_to_string())
with open("salenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults
- scikit-learn



#### Create Azure ML Scoring file

#We also need to tell how to score our training model
#Here we are creating scoring model and also sending it to docker image


In [28]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('sal_model')
    # the most recent one "sal_model"
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Overwriting score.py


#### Deploy the model to Azure Container Instance


Now take the model, The Env file and scoring file -> Turn them into a docker image

https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.image.container.containerimage?view=azure-ml-py

In [32]:
%%time
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="salenv.yml")

Wall time: 0 ns


So basically we are generating score.py and salenv.yml file from code above within the jupyter notebook

So instead of me writing Django or flask webservice, we are getting score.py to AzureML and saying that take all these 3 and turn that into a docker image.

#### Expose web service

In [33]:
service = Webservice.deploy_from_model(workspace=ws,
                                       name='salary-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image salary-svc:1, operation "Succeeded"
Creating service
Running...............
SucceededACI service creation operation finished, operation "Succeeded"


Previous step has created an image
#this image is nthing but a docker image 
#this docker image is not containing everything that we have fed 
#-It has serialised model
#-Env to install all the dependencies
#-Then there is code.py
#By the way we can also pull this image locally and run it locally


From above code- it takes the image and then turns it into a ACI and turns that ACI image into web service

#### Get the Web Service URL

Finally we can get the scoring URL which is available as part of our resource group

In [34]:
print(service.scoring_uri)

http://40.119.206.246:80/score


In [35]:
!curl -X POST \
	-H 'Content-Type':'application/json' \
	-d '{"data":[[45]]}' \
	http://40.119.206.56:80/score

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:04 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:05 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:06 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:07 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:08 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:09 --:--:--     0
  0     0    0     0    0     0      0      0 --:--

#### Delete Workspace and clean up resources

In [31]:
ws.delete()